<a href="https://colab.research.google.com/github/JiakangChenBuff/multiproject-2020/blob/main/MultiVarProjectApp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instructions

1. "Compile" the code, under the section labeled "Run this Code!", by clicking the "[ ]", which will turn into an arrow icon after hovering over it, next to the "5 cells hidden" message.
2. Open the Application section, by clicking the arrow next to the word "Application", and then run the application by clicking on the words "linear_widget.widget" and then pressing "Shift+Enter".
3. Play around with the different parameters to better visualize how gradient descent works!

# Run this Code!

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [ ]:
from IPython.display import display

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math

In [ ]:
def PolyCoefficients(x, coeffs):
  o = len(coeffs)
  z = 0
  for i in range(o):
    z += coeffs[i] * x ** i
  return z

def plot_gradient_descent_batch(X, X_b, y, theta, eta):
    m = len(X_b)
    fig = plt.figure(figsize=(20,10))
    ax = plt.axes()
    ax.plot(X, y, "b.")
    n_iterations = 1000
    for iteration in range(n_iterations):
        if iteration < 10:
            style = "b-" if iteration > 0 else "r--"
            x = np.linspace(-2, 2, 100)
            ax.plot(x, PolyCoefficients(x, theta), style, alpha=0.1*(iteration+1))
        gradients = 2/m * X_b.T.dot(X_b.dot(theta) - y)
        theta = theta - eta * gradients
    plt.xlabel("$x_1$", fontsize=18)
    plt.title(r"$\eta = {}$".format(eta), fontsize=16)
    plt.show()

def learning_schedule(t_0, t_1, t):
  return t_0 / (t + t_1)

def plot_gradient_descent_stochastic(X, X_b, y, theta, t_0, t_1):
  n_epochs = 50
  m = len(X_b)
  fig = plt.figure(figsize=(20,10))
  ax = plt.axes()
  for epoch in range(n_epochs):
    for i in range(m):
      if epoch == 0 and i < 20:
        style = "b-" if i > 0 else "r--"
        x = np.linspace(-2, 2, 100)
        ax.plot(x, PolyCoefficients(x, theta), style, alpha=0.1*(i / 2))
      random_index = np.random.randint(m)
      xi = X_b[random_index:random_index+1]
      yi = y[random_index:random_index+1]
      gradients = 2 * xi.T.dot(xi.dot(theta) - yi)
      eta = learning_schedule(t_0, t_1, epoch * m + i)
      theta = theta - eta * gradients
  plt.plot(X, y, "b.")
  plt.xlabel("$x_1$", fontsize=18)
  plt.show()


def plot_gradient_descent_mini(X, X_b, y, theta, t_0, t_1, minibatch_size):
  t = 0
  n_iterations = 50
  m = len(X_b)
  fig = plt.figure(figsize=(20,10))
  ax = plt.axes()
  for epoch in range(n_iterations):
    shuffled_indices = np.random.permutation(m)
    X_b_shuffled = X_b[shuffled_indices]
    y_shuffled = y[shuffled_indices]
    for i in range(0, m, minibatch_size):
      if t < 15:
        style = "b-" if t != 0 else "r--"
        x = np.linspace(-2, 2, 100)
        ax.plot(x, PolyCoefficients(x, theta), style, alpha=(1/15)*t)
      t += 1
      xi = X_b_shuffled[i: i + minibatch_size]
      yi = y_shuffled[i:i+minibatch_size]
      gradients = 2/minibatch_size * xi.T.dot(xi.dot(theta) - yi)
      eta = learning_schedule(t_0, t_1, t)
      theta = theta - eta * gradients
  plt.plot(X, y, "b.")
  plt.xlabel("$x_1$", fontsize=18)
  plt.show()


def visualize(function, alg, constant, x_coefficient, x_sq_coefficient, x_cube_coef, step_size, t_0, t_1, batch_size):

  X = 4 * np.random.rand(100, 1) - 2
  y = constant + x_coefficient * X + x_sq_coefficient * X * X + x_cube_coef * X * X * X + 0.1*np.random.randn(100, 1)
  if (function == 'Linear'):
    X_b = np.c_[np.ones((100, 1)), X]
    theta = np.random.randn(2, 1)
  elif (function == 'Quadratic'):
    X_b = np.c_[np.ones((100, 1)), X, X * X]
    theta = np.random.randn(3, 1)
  else:
    X_b = np.c_[np.ones((100, 1)), X, X * X, X * X * X]
    theta = np.random.randn(4, 1)
  if (alg == 'Batch Gradient Descent'):
    plot_gradient_descent_batch(X, X_b, y, theta, step_size)
  elif (alg == 'Stochastic Gradient Descent'):
    plot_gradient_descent_stochastic(X, X_b, y, theta, t_0, t_1)
  else:
    plot_gradient_descent_mini(X, X_b, y, theta, t_0, t_1, batch_size)

func_dd = widgets.Dropdown(options=['--Select a Function--', 'Linear', 'Quadratic', 'Cubic'], value='--Select a Function--', description='Function: ')
dropdown = widgets.Dropdown(options=['--Select an Algorithm--', 'Batch Gradient Descent', 'Stochastic Gradient Descent', 'Mini-Batch Gradient Descent'], value='--Select an Algorithm--', description='Type: ')
const_slider = widgets.FloatSlider(value=0,min=-1,max=1,step=0.1, description='Constant: ', disabled=True)
lin_slider = widgets.FloatSlider(value=0,min=-1,max=1,step=0.1, description='X Coef.: ', disabled=True)
sq_slider = widgets.FloatSlider(value=0,min=-1,max=1,step=0.1, description='X^2 Coef.: ', disabled=True)
cube_slider = widgets.FloatSlider(value=0,min=-1,max=1,step=0.1, description='X^3 Coef.: ', disabled=True)
eta = widgets.FloatText(value=0.1,description='Step Size: ', disabled=True)
learning_rate_1 = widgets.FloatText(value=5,description='t_0: ', disabled=True)
learning_rate_2 = widgets.FloatText(value=50, description='t_1: ', disabled=True)
mini_batch = widgets.IntText(value=20, description = 'Mini-Batch: ', disabled=True)


const_slider.style.handle_color = 'lightgray'
lin_slider.style.handle_color = 'lightgray'
sq_slider.style.handle_color = 'lightgray'
cube_slider.style.handle_color = 'lightgray'

def disable_coefficients(*args):
  if (func_dd.value == 'Linear'):
    const_slider.disabled=False
    const_slider.value = 0
    const_slider.style.handle_color = 'deepskyblue'
    lin_slider.disabled=False
    lin_slider.value = 0
    lin_slider.style.handle_color = 'deepskyblue'
    sq_slider.disabled=True
    sq_slider.value = 0
    sq_slider.style.handle_color = 'lightgray'
    cube_slider.disabled=True
    cube_slider.value = 0
    cube_slider.style.handle_color = 'lightgray'
  elif (func_dd.value == 'Quadratic'):
    const_slider.disabled=False
    const_slider.value = 0
    const_slider.style.handle_color = 'deepskyblue'
    lin_slider.disabled=False
    lin_slider.value = 0
    lin_slider.style.handle_color = 'deepskyblue'
    sq_slider.disabled=False
    sq_slider.value = 0
    sq_slider.style.handle_color = 'deepskyblue'
    cube_slider.disabled=True
    cube_slider.value = 0
    cube_slider.style.handle_color = 'lightgray'
  elif (func_dd.value == '--Select a Function--'):
    const_slider.disabled=True
    const_slider.value = 0
    const_slider.style.handle_color = 'lightgray'
    lin_slider.disabled=True
    lin_slider.value = 0
    lin_slider.style.handle_color = 'lightgray'
    sq_slider.disabled=True
    sq_slider.value = 0
    sq_slider.style.handle_color = 'lightgray'
    cube_slider.disabled=True
    cube_slider.value = 0
    cube_slider.style.handle_color = 'lightgray'
  else:
    const_slider.disabled=False
    const_slider.value = 0
    const_slider.style.handle_color = 'deepskyblue'
    lin_slider.disabled=False
    lin_slider.value = 0
    lin_slider.style.handle_color = 'deepskyblue'
    sq_slider.disabled=False
    sq_slider.value = 0
    sq_slider.style.handle_color = 'deepskyblue'
    cube_slider.disabled=False
    cube_slider.value = 0
    cube_slider.style.handle_color = 'deepskyblue'

def disable_stuff(*args):
  if (dropdown.value == 'Batch Gradient Descent'):
    eta.disabled=False
    learning_rate_1.disabled=True
    learning_rate_2.disabled=True
    mini_batch.disabled=True
  elif (dropdown.value == 'Stochastic Gradient Descent'):
    eta.disabled=True
    learning_rate_1.disabled=False
    learning_rate_2.disabled=False
    mini_batch.disabled=True
  elif (dropdown.value == '--Select an Algorithm--'):
    eta.disabled=True
    learning_rate_1.disabled=True
    learning_rate_2.disabled=True
    mini_batch.disabled=True
  else:
    eta.disabled=True
    learning_rate_1.disabled=False
    learning_rate_2.disabled=False
    mini_batch.disabled=False

func_dd.observe(disable_coefficients, 'value')
dropdown.observe(disable_stuff, 'value')

linear_widget = interact_manual(visualize, 
                          function=func_dd,
                          alg=dropdown,
                          constant=const_slider,
                         x_coefficient=lin_slider,
                         x_sq_coefficient=sq_slider,
                         x_cube_coef=cube_slider,
                         step_size=eta,
                         t_0=learning_rate_1,
                         t_1=learning_rate_2,
                         batch_size=mini_batch)

interactive(children=(Dropdown(description='Function: ', options=('--Select a Function--', 'Linear', 'Quadrati…

# Application

In [ ]:
linear_widget.widget